In [7]:

import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

#duckduckgo 차단당해서 뺌 . 
# class StockMarketSymbolSearchToolArgsSchema(BaseModel):
#     query: str = Field(
#         description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
#     )


# class StockMarketSymbolSearchTool(BaseTool):
#     name = "StockMarketSymbolSearchTool"
#     description = """
#     Use this tool to find the stock market symbol for a company.
#     It takes a query as an argument.
    
#     """
#     args_schema: Type[
#         StockMarketSymbolSearchToolArgsSchema
#     ] = StockMarketSymbolSearchToolArgsSchema

#     def _run(self, query):
#         ddg = DuckDuckGoSearchAPIWrapper()
#         return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        #StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CompanyOverview` with `{'symbol': 'NET'}`


{'Symbol': 'NET', 'AssetType': 'Common Stock', 'Name': 'Cloudflare Inc', 'Description': 'CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world. The company is headquartered in San Francisco, California.', 'CIK': '1477333', 'Exchange': 'NYSE', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'SOFTWARE - INFRASTRUCTURE', 'Address': '101 TOWNSEND STREET, SAN FRANCISCO, CA, UNITED STATES, 94107', 'OfficialSite': 'https://www.cloudflare.com', 'FiscalYearEnd': 'December', 'LatestQuarter': '2025-12-31', 'MarketCapitalization': '66650919000', 'EBITDA': '-48686000', 'PERatio': 'None', 'PEGRatio': '2.355', 'BookValue': '4.147', 'DividendPerShare': 'None', 'DividendYield': 'None', 'EPS': '-0.28', 'RevenuePerShareTTM': '5.81', 'ProfitMargin': '-0.0512', 'OperatingMarginTTM': '-0.0667', 'ReturnOnAssetsTTM': '-0.0271', 'Ret

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 24352 tokens (24153 in the messages, 199 in the functions). Please reduce the length of the messages or functions.